<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Вызов функций с помощью помощников - упражнение  

**ОБЩЕЕ ЗАДАНИЕ: учитывая функцию, показанную ниже, создайте помощника, который сможет создавать аргументы для функции, отправлять их обратно через сообщение, а затем запускать функцию, чтобы получить пользовательский ввод и отправить эту информацию обратно помощнику, а затем получить от помощника окончательный ответ о проверке ответа**

In [1]:
from openai import OpenAI
client = OpenAI()

## Удаление всех ассистентов

----

In [2]:
def delete_all_assistants(client):
    """
    Удаляет всех ассистентов, связанных с вашим аккаунтом OpenAI.

    Args:
        client: Экземпляр клиента OpenAI.
    """

    assistants = client.beta.assistants.list()  # Получаем список всех ассистентов

    for assistant in assistants.data:
        assistant_id = assistant.id
        print(f"Удаление ассистента: {assistant_id}")
        try:
            client.beta.assistants.delete(assistant_id)  # Удаляем ассистента
        except Exception as e:
            print(f"Ошибка при удалении ассистента {assistant_id}: {e}")


In [ ]:
delete_all_assistants(client)

**ЗАДАНИЕ: Просмотрите приведенную ниже функцию и убедитесь, что вы ее поняли.**

In [23]:
def president_home_state(president,state_options):
    '''
    INPUTS:
        president str = Строковое имя президента США
        state_options list[str] = Список возможных штатов, в которых родился президент, причем один из них должен быть верным!
    
    OUTPUTS:
        response str = Ответ, который пользователь выбрал в качестве правильного штата места рождения.
    
    '''
    print("Привет! Давайте проверим ваши знания о штатах, где родились президенты США!")
    print(f"В каком штате родился этот президент: {president}\n")
    
    for num,option in enumerate(state_options):
        print('\n')
        print(f"Определение #{num} это: {option}")
    
    print('\n')
    num_choice = input("Каков ваш выбор? (Верните вариант с одним числом)")
    
    return state_options[int(num_choice)]
    

In [24]:
president_home_state("George Washington", ["Virginia", "New York", "Massachusetts"])


Привет! Давайте проверим ваши знания о штатах, где родились президенты США!
В каком штате родился этот президент: George Washington



Определение #0 это: Virginia


Определение #1 это: New York


Определение #2 это: Massachusetts




'New York'

**TASK: Convert the Python function above into a JSON Description. Here is a general guide to help you:**

     tool = {'type':'function',
                    'function':{
                        'name': 'my_function_name',
                        'parameters':{
                            "type":"object",
                              "properties":{
                                  "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
                                  "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
                              },
                            'required' : ['parameter_one','parameter_two']
                        }

            }
        } 

In [3]:
# CODE HERE
function_json = {
            'type':'function',
            'function':{
                'name': 'president_home_state',
                'description': 'Функция создает интерактивный тест с вопросом о месте рождения президента США и предоставляет список вариантов ответов для выбора.',
                'parameters':{
                    "type":"object",
                    "properties":{
                        'president': {
                            "type": "string",
                            "description": "Имя бывшего или действующего президента США, о котором задается вопрос в тесте.",
                        },
                        "state_options": {
                            'type':'array', 
                                "items":{'type':'string'},
                                'description':"Список возможных штатов, в которых мог родиться президент. Один из вариантов должен быть правильным."}
                        },
                    'required' : ["president","state_options"]
                }
            }
        }
                
                        

**TASK: Create the OpenAI API Client Connection**

In [4]:
assistant = client.beta.assistants.create(
    model="gpt-3.5-turbo-0125",
    name="Помощник в составлении тестов для проверки знаний людей",
    description="Тестовый бот, для вызова функций",
    instructions="Вы помогаете создать тест для проверки определений слов, предоставляя слово, а затем несколько вариантов определений, где только один вариант является правильным. Пишите только на РУССКОМ ЯЗЫКЕ",
    temperature=0,
    tools=[function_json]
)

**TASK: Create the Assistant with the Function Calling Tool**

In [31]:
run = client.beta.threads.create_and_run(
    assistant_id=assistant.id,
    thread = {
    "messages": [
        {
            "role": "user",  # "user" or "assistant"
            "content": "Создайте новый список слов и определений вопроса викторины. Затем сообщите мне, является ли полученный ответ правильным.",  # Text content of the message
            "attachments": [],  # (Optional) List of file attachments
        }
        # Add more message objects as needed
    ]
    }
    
)


In [32]:
run

Run(id='run_uKvSjHACINqtNAeZtANTepd9', assistant_id='asst_yEqI1lwfIUVkrRAJ114rfd7e', cancelled_at=None, completed_at=None, created_at=1716027194, expires_at=1716027794, failed_at=None, incomplete_details=None, instructions='Вы помогаете создать тест для проверки определений слов, предоставляя слово, а затем несколько вариантов определений, где только один вариант является правильным. Пишите только на РУССКОМ ЯЗЫКЕ', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Bvko9KrC4tFVCm34TitlDPSG', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='president_home_state', description='Функция создает интерактивный тест с вопросом о месте рождения президента США и предоставляет список вариантов ответов для выбора.', parameters={'type': 'object', 'properties': {'president': {'type': 'string', 'des

In [33]:
thread_id="thread_Bvko9KrC4tFVCm34TitlDPSG"

**TASK: Create a new thread.**

In [34]:
# CODE HERE
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread,
            run_id=run.id,
        )
        print(run.status)
        time.sleep(0.5)
    return run

In [35]:
run = wait_on_run(run,thread_id)

requires_action


In [19]:
run

Run(id='run_LoJImShTDayez14zqAKAHH4e', assistant_id='asst_yEqI1lwfIUVkrRAJ114rfd7e', cancelled_at=None, completed_at=None, created_at=1716026205, expires_at=1716026805, failed_at=None, incomplete_details=None, instructions='Вы помогаете создать тест для проверки определений слов, предоставляя слово, а затем несколько вариантов определений, где только один вариант является правильным. Пишите только на РУССКОМ ЯЗЫКЕ', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_yuMEcBM0oxhnZfiK1oY2BE6t', function=Function(arguments='{"president": "Джордж Вашингтон", "state_options": ["Вирджиния", "Техас", "Флорида", "Калифорния"]}', name='president_home_state'), type='function'), RequiredActionFunctionToolCall(id='call_xio6HG7PsAYW4twmG2QHzZUz', function=Function(arguments='{"president"

**Задание: Создайте новое сообщение с запросом на нового президента США и список вариантов штатов. Затем сообщите модели, что позже отправите ответ, и попросите помощника проверить правильность сообщения**.

In [36]:
import json

# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[1]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

AttributeError: 'list' object has no attribute 'function'

In [25]:
answer = president_home_state(arguments['president'],arguments['state_options'])

Привет! Давайте проверим ваши знания о штатах, где родились президенты США!
В каком штате родился этот президент: Джордж Вашингтон



Определение #0 это: Вирджиния


Определение #1 это: Техас


Определение #2 это: Флорида


Определение #3 это: Калифорния




**TASK: Run the thread with the assistant.**

In [26]:
answer

'Техас'

In [ ]:
import json

tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_outputs = []

# Обрабатываем каждый вызов функции
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    
    # Вызываем функцию и получаем результат
    result = president_home_state(**function_args) 
    
    # Получаем ответ пользователя и проверяем его
    user_answer = input("Ваш ответ: ")
    correct_answer = function_args["state_options"][0] 
    is_correct = user_answer.lower() == correct_answer.lower()
    
    tool_outputs.append(
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps({"answer": result, "is_correct": is_correct}), 
        }
    )

# Отправляем результаты всех вызовов функций
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread_id,
    run_id=run.id,
    tool_outputs=tool_outputs
)


In [39]:
tool_calls

[RequiredActionFunctionToolCall(id='call_YevN3L96mK0Fv1Idopl9gTM7', function=Function(arguments='{"president": "Джордж Вашингтон", "state_options": ["Вирджиния", "Техас", "Флорида", "Калифорния"]}', name='president_home_state'), type='function'),
 RequiredActionFunctionToolCall(id='call_a2Jiiisasy8emYfP27zclsTF', function=Function(arguments='{"president": "Авраам Линкольн", "state_options": ["Иллинойс", "Огайо", "Пенсильвания", "Мичиган"]}', name='president_home_state'), type='function'),
 RequiredActionFunctionToolCall(id='call_AgHWcC1CYAz572GqJqMrmhFz', function=Function(arguments='{"president": "Джон Кеннеди", "state_options": ["Массачусетс", "Вашингтон", "Нью-Йорк", "Техас"]}', name='president_home_state'), type='function')]

In [38]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread_id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(answer),
        }
    ],
)

BadRequestError: Error code: 400 - {'error': {'message': "Expected tool outputs for call_ids ['call_YevN3L96mK0Fv1Idopl9gTM7', 'call_a2Jiiisasy8emYfP27zclsTF', 'call_AgHWcC1CYAz572GqJqMrmhFz'], got ['call_AgHWcC1CYAz572GqJqMrmhFz']", 'type': 'invalid_request_error', 'param': None, 'code': None}}

**TASK: Confirm that the run.status requires action**

In [5]:
#CODE HERE

'requires_action'

**TASK: Retrieve the Run result arguments for the tool and pass them into the president_home_state() function**

In [6]:
#CODE HERE

Function Name: president_home_state
Function Arguments:


{'president': 'Thomas Jefferson',
 'state_options': ['Virginia', 'New York', 'California', 'Texas']}

Hello! Let's test your knowledge of the home states of US Presidents!
In what state was this president born: Thomas Jefferson



Definition #0 is: Virginia


Definition #1 is: New York


Definition #2 is: California


Definition #3 is: Texas




What is your choice? (Return the single number option) 0


'Virginia'

**TASK: Return the local function response to the tool in the assistant.**

In [72]:
#CODE HERE

'completed'

**TASK: Retrieve the Messages and see if you got the state correct!**

In [76]:
#CODE HERE

Yes, Thomas Jefferson was indeed born in Virginia. Well done!


Was Thomas Jefferson born in Virginia?


Create a new quiz question with a US President and a list of options for the home state of birth. Then I will reply with a single state and let me know if I got it right.




**TASK: Delete the Assistant**

In [ ]:
#CODE HERE